<a href="https://colab.research.google.com/github/nickprock/corso_data_science/blob/devs/machine_learning_pills/01_supervised/01_california_housing_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## California Housing Price

<br>

![housing](https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fstatic1.businessinsider.com%2Fimage%2F5b2a86cf1ae6624a008b5492-1190-625%2Fcalifornias-housing-market-has-reached-a-boiling-point-and-a-typical-home-costs-600000.jpg&f=1&nofb=1)

<br>

[Image Credits](https://www.businessinsider.com/california-home-price-hits-record-high-2018-6?IR=T)

<br>

Il primo esempio che vedremo è abbastanza noto in letteratura.

**Date alcune caratteristiche delle case stimare il prezzo (la mediana in questo caso**.

Il modello adottato è una regressione lineare:
* i dati sono etichettati, ogni esempio ha il prezzo della casa, quindi siamo nel contesto dell' **approccio supervisionato**
* la nostra **variabile target è numerica continua** quindi adottiamo la regressione

Man mano verranno spiegate tutte le fasi del processo di definizione del progetto di Machine Learning (nei prossimi le varie fasi saranno molto meno approfondite).

Questo notebook si basa sull'esempio al capitolo 2 di [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/).

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Dataset

Il dataset *California Housing Price* è stato usato nel 1997 per l'articolo *"Sparse Spatial Autoregressions" di Pace, R. Kelley and Ronald Barry, pubblicato in Statistics and Probability Letters.*

Contiene 20640 osservazioni (poche per i modelli in produzione ma ottime per un esercizio) del 1990, si basa sul concetto di **block group** che è un'area geografica che và da circa 600 a 3000 persone utilizzato dal Census Bureau U.S., nel notebook il block groupè verrà chiamato *district*.

Le variabili presenti:
* latitudine
* longitudine
* housing median age
* total rooms: il numero di camere nel distretto
* total bedrooms: il numero totale di camere da letto nel distretto
* population: la popolazione nel distretto
* households: numero di famiglie nel distretto
* median_income: reddito medio nel distretto
* **median_house_value**
* ocean_proximity: vicinanza all'oceano

### Caricamento del dataset

***Se stai usando il notebook su Colab esegui le prossime due celle, altrimenenti vai direttamente al caricamento con *read_csv* inserendo il path del tuo file *housing.csv***

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/ENTER_YOUR_CODE'
fluff, id = link.split('=')

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('housing.csv')

In [0]:
df = pd.read_csv("housing.csv")
df.head()

### Analisi del dataset

Ogni riga rappresenta un distretto e ha dieci attributi.

La prima cosa che si nota è che sono tutte variabili numeriche tranne ocean_proximity, stampiamo le informazioni sul dataset per non incorrere in eventuali errori (es. valori numerici che in realtà sono stringhe)

In [0]:
df.info()

Dalle info notiamo anche che la variabile total_bedrooms ha dei valori mancanti, sono una piccola percentuale in seguito vedremo come trattarli.

Vediamo:
* la distribuzione dei valori per ocean_proximity
* le statistiche riassuntive per tutte le variabili numeriche

In [0]:
df.ocean_proximity.value_counts()

In [0]:
plt.figure(figsize=(18,10))
plt.bar(np.arange(len(df.ocean_proximity.value_counts())),df.ocean_proximity.value_counts())
plt.xticks(np.arange(len(df.ocean_proximity.value_counts())), df.ocean_proximity.value_counts().index)

In [0]:
df.describe()

In [0]:
df.hist(bins=50, figsize=(20, 15))

Annotazione:
* la variabile median_income non è espressa in dollari ma và da 0,5 a 15
* sia l'età che il prezzo delle case raggiungono un massimo oltre il quale non vanno, quindi il modello non prevederà un prezzo oltre quel valore
* gli attributi hanno scala molto differente tra loro
* Guardando gli istogrammi si nota che molte variabili hanno una coda lunga, la distribuzione è tutta schiacciata a sinistra con una coda a destra molto pesante

## Scikit-learn

<br>

![sklearn](https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png)

<br>

[Image Credits](https://scikit-learn.org/stable/index.html)

<br>

Da qui in avanti useremo alcuni moduli e funzioni presenti nella libreria **scikit-learn**.

Racchiude moduli per sviluppare un processo di machine learning, è semplice ed efficiente, basata (come pandas) su numpy e scipy, contiene modelli di:
* regressione
* classificazione
* clustering

La libreria è open source ([Licenza BSD](https://it.wikipedia.org/wiki/Licenze_BSD)) quindi utilizzabile ed estendibile.

Per richiamarla in python si può usare il comando:
```
import sklearn
```
anche se solitamente non si fa vista la vastità della libreria, i  metodi complessi che racchiude e il derivante tempo di caricamento che ne comporterebbe. Solitamente si importa la funzione o il modulo di cui si hà bisogno.

### Train - Test Split

Per prima cosa dividiamo il dataset in train e test:
* **train set**: la parte di dati su cui verrà *allenato* il modello. Su questi dati il modello calibrerà i suoi parametri basandosi sugli esempi a disposizione
* **test set**:  la parte dei dati su cui verranno misurate le performance del modello.

Solitamente questa suddivisione può essere 80-20 o 75-25 ma và effettuata dal data scientist a seconda del caso su cui si sta lavorando.

I parametri che imposteremo sono:
* il dataset
* test_size: la percentuale di casi che vanno a finire nel test set
* random_state: un numero che garantisce la replicabilità dell'esercizio, potete scegliere quale volete, cambiando numero cambia la distribuzione delle osservazioni.

*train_test_split* applica un'estrazione casuale senza reimmissione.


In [0]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 42)

print("train_set: ", "\n", train_set)
print("\n")
print("test_set: ", "\n", test_set)

Abbiamo diviso le osservazioni in maniera assolutamente casuale, questa è la strada più semplice e anche quella esatta se ci sono abbastanza osservazioni da rendere la distribuzione dei casi abbastanza uniforme.

Nel caso del nostro dataset non è così, la nostra variabile *median_income* (che è direttamente legata a quella di risposta) è numerica continua e ricade tra quelle con una coda molto lunga, possiamo vederne la distribuzione.

In [0]:
df.median_income.hist(bins=50, figsize=(18,10))
plt.title("Median Income")

Per ovviare a questo problema e avere tutti i livelli compresi sia nel train:
* così il modello può allenarsi su quel caso

sia nel test:
* così possiamo valutare di quanto sbaglia in quel caso

verrà usato il campionamento stratificato. Questa tecnica fa si che ogni campione sia rappresentativo dell'intera popolazione visto che ne riporta le medesime proporzioni.

Essendo la nostra variabile numerica dovrà essere discretizzata e successivamente applicato il campionamento.

In [0]:
df["income_cat"] = pd.cut(df["median_income"], bins=[0.0,1.5,3.0,4.5,6.0,np.inf], labels=[1,2,3,4,5])
df.income_cat.hist()

In [0]:
strat_train_set, strat_test_set = train_test_split(df, test_size = 0.2, random_state = 42, stratify = df.income_cat)

# ora possiamo rimuovere la variabile che non ci serve più
for set_ in (strat_train_set, strat_test_set):
  set_.drop(columns="income_cat", axis=1, inplace=True)

### Analisi della correlazione

Vediamo come sono distribuite geograficamente le nostre abitazioni e successivamente facciamo un'analisi della correlazione delle variabili indipendenti con quella target.

La gradazione del colore, dal blu al rosso è data dal valore, la grandezza del punto dalla densità di popolazione nel blocco.

Questa analisi verrà fatta solo sul train set.

In [0]:
house = strat_train_set.copy()

In [0]:
house.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4, s=house["population"]/100, label="Population", figsize=(18,10),
           c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()

Come si può vedere le case più costose stanno sulla costa come era facilmente prevedibile.

Ora conduciamo un'**analisi di correlazione**.

La correlazione che utilizzeremo è quella di Pearson, si applica a dati numerici continui e rileval la relazione **lineare** che c'è tra le variabili. 
L'indice di correlazione varia tra -1 e 1:
* -1 massima correlazioni negativa
* 1 massima correlazione positiva
* 0 nessuna correlazione

Ci può dire:
* quali sono le variabili che impattano maggiormente sul target
* se ci sono variabili fortemente correlate tra loro e quindi informazioni ridondanti.

In [0]:
corr_matrix = house.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [0]:
# plot delle più correlate
pd.plotting.scatter_matrix(house[["median_house_value", "median_income", "total_rooms", "housing_median_age"]], figsize=(18,10))

La variabile più importante per prevedere il valore mediano della casa è lo stipendio mediano del blocco in cui sorge.

## Cleaning

Inizia la fase di cleaning, per prima cosa ci creiamo una copia del train su cui effettuare la pulizia.

Dividiamo le variabili indipendenti dal target.

In [0]:
house = strat_train_set.drop("median_house_value", axis=1).copy()
house_label = strat_train_set["median_house_value"].copy()

### Imputare i valori mancanti

La prima procedura da fare è decidere come lavorare i valori mancanti, questo è un passaggio fondamentale perchè **qualsiasi strategia scegliamo introduce errore sistematico** nel dataset, l'unica cosa che possiamo fare è scegliere la strategia che lo minimizza.

Strategie che è possibile adottare:
* se la percetuale di valori mancanti è alta eliminare la variabile
* eliminare l'intera osservazione (alta perdita di informazione)
* inserire 0 al posto del valore mancante (alto errore sistematico introdotto)
* sostituire il valore con media, mediana o valore più frequente (strategia consigliata)

Scikit-learn ha funzioni apposite, si chiamano **Imputer** in questo caso non le useremo, ma sostituremo ugualmente i valori mancanti con la mediana della variabile.

In [0]:
# calcoliamo la mediana sul train
median = house["total_bedrooms"].median()
house["total_bedrooms"].fillna(median, inplace = True)
# la variabile mediana ci servirà in seguito per sostiture i valori nel test set, infatti non vengono ricalcolati ma solo imputati

### Convertire le stringhe

La seconda operazione è quella di convertire le stringhe in valori numerici. Ci sono molti metodi di conversione, noi in questo caso useremo il **One-Hot Encoding**.

<br>

![onehot](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fxamlbrewer.files.wordpress.com%2F2019%2F04%2Fone-hot_encoding.png&f=1&nofb=1)

<br>

[Image Credits](https://xamlbrewer.wordpress.com/2019/04/23/machine-learning-with-ml-net-in-uwp-field-aware-factorization-machine/)

<br>

Per ogni osservazione si crea un vettore di lunghezza N pari al numero di categorie nella nostra variabile, ogni elemento avrà valore 0 se l'osservazione non appartiene alla categoria, 1 se appartiene.

***N.B. a differenza di quanto succede nella statistica classica in cui per una variabile categoriale con M attributi bastano M-1 dummies nel machine learning è buona norma tenerle tutte (questa caratteristica si accentua nelle reti neurali).***

In [0]:
house_cat = house[["ocean_proximity"]]

In [0]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

In [0]:
house_cat_onehot = encoder.fit_transform(house_cat)

print(house_cat_onehot.toarray())
print("\n")
print(encoder.categories_)

### Scaling

Gli algoritmi di machine learning non digeriscono bene le variabili numeriche con scala differente tra loro. Per ovviare a questo i metodi più utilizzati sono:
* **min-max** scaling (anche detta normalizzazione) i valori vengono portati in una scala [0,1], scikit-learn permette anche di personalizzare questo intervallo.
* **standard** scaling, ovvero ogni osservazione meno la media e diviso al deviazione standard.

Useremo il secondo approccio.

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [0]:
house.drop("ocean_proximity", axis=1, inplace=True)

In [0]:
house_stand = pd.DataFrame(scaler.fit_transform(house))
house_stand.columns = house.columns
house_stand.head()

Ora dobbiamo concatenare i valori numerici preprocessati con la variabile categorica dopo il onehot encoding.

In [0]:
house_processed = pd.concat([house_stand, pd.SparseDataFrame(house_cat_onehot.toarray(), columns=['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'], default_fill_value=0)], axis=1)

In [0]:
house_processed.head()

## Training and Evaluation

Per risolvere questo problema ho scelto il modello di regressione più semplice, la **regressione lineare**, per approfondire altri modelli consultare i link utili.

<br>

![lr](https://files.realpython.com/media/fig-lin-reg.a506035b654a.png)

<br>

[Image Credits](https://realpython.com/linear-regression-in-python/)

<br>

Non faremo tuning degli iperparametri del modello, ma prenderemo quelli di default.

In [0]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

In [0]:
lr.fit(house_processed, house_label)

Ora abbiamo allenato il modello sul train, dobbiamo preparare il test set per misurare le performance.

In [0]:
test_x = test_set.drop("median_house_value", axis= 1).copy()
test_y = test_set["median_house_value"].copy()

In [0]:
# missing values
test_x["total_bedrooms"].fillna(median, inplace = True)

# one-hot encoding
test_cat_onehot = pd.SparseDataFrame(encoder.transform(test_x[["ocean_proximity"]]).toarray(), columns=['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'], default_fill_value=0)

# scaling
test_x.drop("ocean_proximity", axis=1, inplace=True)

test_x_stand = pd.DataFrame(scaler.transform(test_x))
test_x_stand.columns = test_x.columns

# unione
test_x_processed = pd.concat([test_x_stand, test_cat_onehot], axis=1)

In [0]:
yhat = lr.predict(test_x_processed)

In [0]:
yhat

#### Valutazione del modello

Per valutare un modello di regressione abbiamo due metriche fondamentali:
* errore
* R^2

Per quanto riguarda l'errore può essere utilizzato o la radice dell'errore quadratico medio o l'errore assoluto medio, dipende dai casi, solitamente si utilizza il primo perchè **rafforza** le distanze tra previsione e valore reale, ma in presenza di molti outliers nei dati e da preferire quello assoluto.

Nel nostro caso useremo il RMSE.

<br>

![RMSE](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn-images-1.medium.com%2Fmax%2F1600%2F0*CRZU7qETwW3bwBwK.gif&f=1&nofb=1)

<br>

[Image Credit](https://towardsdatascience.com/understanding-objective-functions-in-neural-networks-d217cb068138)

<br>

In [0]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(test_y, yhat))
print(rmse)

Questo non è un grande risultato, vuol dire che il nostro modello sbaglia in media di quasi 70K$, siamo in un caso di **underfitting**.

Si dice che un modello và in **underfitting**:
* quando i dati non sono abbastanza esplicativi per il fenomeno
* quando le ipotesi alla base del modello sono troppo semplici e quest'ultimo non riesce a catturare i pattern nei dati.

Al contrario un modello và in **overfitting** quando non riesce a generalizzare e si adatta troppo bene al training set.

<br>

![fit](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F1200%2F1*_7OPgojau8hkiPUiHoGK_w.png&f=1&nofb=1)

<br>

[Image Credits](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76)

<br>

#### Cosa si può fare per migliorare la previsione?

Non potendo aumentare il campione di dati possiamo solo usare modelli più potenti e cercare di fare un tuning più preciso.

Per il corso, alla fine delle lezioni potete:

1) Provare questi due modelli:
  * Random Forest Regressor
  * XGBoost Regressor

2) Scegliere i valori degli iperparametri tramite cross-validation

### Link utili

[Repo Github del libro di Aurélien Gèron](https://github.com/ageron/handson-ml2)

[California Housing Price su Kaggle](https://www.kaggle.com/harrywang/housing#housing.csv)

[Underfitting and Overfitting](https://medium.com/greyatom/what-is-underfitting-and-overfitting-in-machine-learning-and-how-to-deal-with-it-6803a989c76)

[Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html#impute)